<a href="https://colab.research.google.com/github/Rajaanthonysamy/my_finetune/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM , Embedding, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
max_len = 200
vocab_size = 10000
embedding_dim = 128
latent_dim = 256

In [ ]:
(x_train, y_train),_ = tf.keras.datasets.imdb.load_data(num_words=vocab_size)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
x_train = x_train[:3000]
y_train = y_train[:3000]

In [ ]:
len(x_train)

3000

In [ ]:
len(y_train)

3000

In [ ]:
input_layer = Input(shape=(max_len,))
embedding_layer = Embedding(vocab_size, embedding_dim)(input_layer)
lstm_layer, state_h,sate_c = LSTM(latent_dim, return_state=True)(embedding_layer)
output_layer = Dense(1, activation='sigmoid')(state_h)
classification_model = Model(input_layer, output_layer)

In [ ]:
classification_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 200, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ [(None, 256), (None,   │       394,240 │
│                                 │ 256), (None, 256)]     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,674,497 (6.39 MB)

 Trainable params: 1,674,497 (6.39 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
classification_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
x_train = pad_sequences(x_train, maxlen=max_len,truncating='post',padding='post')
classification_model.fit(x_train, y_train, epochs=1, batch_size=64,validation_split=0.1)

43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.5538 - loss: 0.6835 - val_accuracy: 0.4467 - val_loss: 0.7002


In [ ]:
sample_review = x_train[0].reshape(1,-1)

pred = classification_model.predict(sample_review)

print(f"Positive (prediction: {pred[0][0]:.4f})"if pred[0][0] >0.5 else f"Negative (prediction: {pred[0][0]:.4f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Positive (prediction: 0.5920)


In [ ]:
word_index = tf.keras.datasets.imdb.get_word_index()
reverse_word_index = dict([(value + 3, key) for (key, value) in word_index.items()])
reverse_word_index[0] = "<PAD>"
reverse_word_index[1] = "<START>"
reverse_word_index[2] = "<UNK>"
reverse_word_index[3] = "<UNUSED>"

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

sample_review_decoded = decode_review(x_train[0])


1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
sample_review = x_train[0].reshape(1,-1)

pred = classification_model.predict(sample_review)

sample_review_decoded = decode_review(x_train[0])
print(f"Review: {sample_review_decoded}")
print(f"Prediction: {"Positive" if pred[0][0] > 0.5 else "Negative"} (score: {pred[0][0]:.4f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Review: <START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these chil